In [ ]:
from keras.preprocessing import image
from keras.models import load_model 
from keras import models, layers
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np 
import random 
import PIL.Image as pilimg 
import imghdr
import pandas as pd 
import pickle 
import keras 
import os
import shutil # 디렉토리 만들기, 

In [ ]:
def rename_images_in_class_folder(src_class_dir, class_name, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)

    files = [f for f in os.listdir(src_class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    files.sort()

    for idx, fname in enumerate(files):
        src_path = os.path.join(src_class_dir, fname)
        ext = os.path.splitext(fname)[1].lower()
        new_name = f"{class_name}.{idx}{ext}"
        dst_path = os.path.join(dest_dir, new_name)
        shutil.copyfile(src_path, dst_path)

    print(f"✅ {class_name} 리네임 완료: {len(files)}개 처리됨.")

In [ ]:
def rename_all_classes(original_root_dir, renamed_root_dir):
    classes = ["daisy", "dandelion", "rose", "sunflower", "tulip"]

    if os.path.exists(renamed_root_dir):
        shutil.rmtree(renamed_root_dir)
    os.makedirs(renamed_root_dir, exist_ok=True)

    for class_name in classes:
        src_class_dir = os.path.join(original_root_dir, class_name)
        rename_images_in_class_folder(src_class_dir, class_name, renamed_root_dir)

In [ ]:
def copy_images_by_class(class_name, original_dataset_dir, dest_dirs, split_ratio=(0.5, 0.25, 0.25)):
    image_files = [
        f for f in os.listdir(original_dataset_dir)
        if f.startswith(f"{class_name}.") and f.lower().endswith(('.jpg', '.jpeg', '.png')) and os.path.isfile(os.path.join(original_dataset_dir, f))
    ]
    image_files.sort()
    random.shuffle(image_files)

    total = len(image_files)
    train_end = int(total * split_ratio[0])
    val_end = train_end + int(total * split_ratio[1])

    splits = [image_files[:train_end], image_files[train_end:val_end], image_files[val_end:]]

    for split, dst_dir in zip(splits, dest_dirs):
        os.makedirs(dst_dir, exist_ok=True)
        for fname in split:
            src = os.path.join(original_dataset_dir, fname)
            dst = os.path.join(dst_dir, fname)
            shutil.copyfile(src, dst)

In [ ]:
def ImageCopy(renamed_dataset_dir, base_dir):
    categories = ["daisy", "dandelion", "rose", "sunflower", "tulip"]
    sets = ["train", "validation", "test"]

    if os.path.exists(base_dir):
        shutil.rmtree(base_dir)
    for set_name in sets:
        for category in categories:
            os.makedirs(os.path.join(base_dir, set_name, category), exist_ok=True)

    train_dir = os.path.join(base_dir, "train")
    val_dir = os.path.join(base_dir, "validation")
    test_dir = os.path.join(base_dir, "test")

    for category in categories:
        print(f"🔄 {category} 분할 중...")
        copy_images_by_class(
            class_name=category,
            original_dataset_dir=renamed_dataset_dir,
            dest_dirs=[
                os.path.join(train_dir, category),
                os.path.join(val_dir, category),
                os.path.join(test_dir, category)
            ],
            split_ratio=(0.5, 0.25, 0.25)
        )

    print("\n✅ 이미지 분할 복사 완료!\n")

    for set_name in sets:
        for category in categories:
            dir_path = os.path.join(base_dir, set_name, category)
            count = len(os.listdir(dir_path))
            print(f"📁 {set_name}/{category}: {count}개")

In [ ]:
# 원본 데이터셋 폴더 (클래스별 하위 폴더 있음)
original_dataset_dir = "C:/Users/Admin/Documents/GitHub/ML-DL-study/deeplearning_workspace/data/flowers"

# 리네임된 이미지들이 저장될 위치
renamed_root = "C:/Users/Admin/Documents/GitHub/ML-DL-study/deeplearning_workspace/data/flowers_renamed"

# 최종 분할된 train/val/test 폴더 생성 위치
base_dir = "C:/Users/Admin/Documents/GitHub/ML-DL-study/deeplearning_workspace/data/flowers_small"

# 1단계: 클래스별 이미지들을 daisy.0.jpg 형식으로 리네임 + 통합
rename_all_classes(original_dataset_dir, renamed_root)

# 2단계: 리네임된 이미지를 2:1:1 비율로 train/validation/test 분할 복사
ImageCopy(renamed_root, base_dir)

In [ ]:
def deeplearning():
    data_augmentation = keras.Sequential([
        layers.RandomFlip("horizontal", input_shape=(180, 180, 3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ])

    model = models.Sequential()
    model.add(layers.Rescaling(1./255))
    model.add(data_augmentation)
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Conv2D(64, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(5, activation="softmax"))  # 👈 클래스 수 = 5

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",        # 👈 정수 라벨용 다중 분류
        metrics=["accuracy"]
    )

    train_ds = keras.utils.image_dataset_from_directory(
        train_dir,
        validation_split=0.2,
        seed=123,
        subset="training",
        image_size=(180, 180),
        batch_size=16
    )

    validation_ds = keras.utils.image_dataset_from_directory(
        validation_dir,
        validation_split=0.2,
        seed=123,
        subset="validation",
        image_size=(180, 180),
        batch_size=16
    )

    model.fit(train_ds, 
              validation_data=validation_ds,
              epochs=30)

    model.save("flowers_model.keras")

deeplearning()

In [ ]:
import os, shutil
import pickle 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
# 이미지 전처리 유틸리티 모듈
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model 
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers

import numpy as np 
import os 
import random 
import PIL.Image as pilimg 
import imghdr
import pandas as pd 
import tensorflow as tf 
import keras 

def study():
    batch_size=32 #한번에 불러올 이미지 개수 
    img_height = 180 #내가 지정한 크기로 이미지를 가져온다  
    img_width  = 180 

    #데이터 증강을 위한 파라미터 지정하기 
    data_augmentation = keras.Sequential(
            [
                    layers.RandomFlip("horizontal"),
                    layers.RandomRotation(0.1),
                    layers.RandomZoom(0.1),
            ]
    )

    model = models.Sequential() 
    model.add(data_augmentation) 
    #증강에 대한 파라미터를 주면 1에포크마다 데이터를 조금씩 변형을 가해서 가져간다
    #과대적합을 막기 위해서 
    model.add(layers.Rescaling(1./255))  #1/255 파이썬 3에서는 결과가 실수 2에서는 정수
    model.add(layers.Conv2D(32, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    
    model.add(layers.Flatten()) #CNN과 완전연결망을 연결한다.
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))
    
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',  #라벨원한코딩안하려고 
                  metrics=['accuracy'])
    
    train_dir = "C:/Users/Admin/Documents/GitHub/ML-DL-study/deeplearning_workspace/data/flowers"
    train_ds = keras.preprocessing.image_dataset_from_directory( 
        train_dir, 
        validation_split=0.2,
        subset='training', 
        seed=1234,  
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    val_ds = keras.preprocessing.image_dataset_from_directory( 
        train_dir, 
        validation_split=0.2,
        subset='validation', #전체 데이터를 20% 나눠서 검증셋으로  
        seed=1234,  
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    history = model.fit(train_ds, 
              epochs=5,
              validation_data = val_ds)
    #모델을 저장하자 
    model.save("flowers_model.keras")
    with open("flowers_hist.hist", "wb") as f:
        pickle.dump(history.history, f)

    return history.history  # 💡 수정된 부분


def drawChart(history):  # ← 인자 받도록 수정
    if not history:
        print("⚠️ 학습 기록이 없습니다.")
        return

    acc = history["accuracy"]
    val_acc = history["val_accuracy"]
    loss = history["loss"]
    val_loss = history["val_loss"]

    epochs = range(len(acc))

    plt.plot(epochs, acc, "bo", label="Training acc")
    plt.plot(epochs, val_acc, "b", label="Validation acc")
    plt.title("Training and validation accuracy")
    plt.legend()

    plt.figure()
    plt.plot(epochs, loss, "bo", label="Training loss")
    plt.plot(epochs, val_loss, "b", label="Validation loss")
    plt.title("Training and validation loss")
    plt.legend()

    plt.show()


def predict():
    # 모델 불러오기
    model = load_model("flowers_model.keras")

    test_dir = "C:/Users/Admin/Documents/GitHub/ML-DL-study/deeplearning_workspace/data/flowers_small/test"
    test_ds = keras.preprocessing.image_dataset_from_directory(
        test_dir,
        validation_split=0.2,
        subset="validation",
        seed=1234,
        image_size=(180, 180),
        batch_size=16
    )

    print("✅ 예측 시작...")

    match_cnt = 0
    total_cnt = 0

    for images, labels in test_ds:
        output = model.predict(images)
        preds = np.argmax(output, axis=1)
        match_cnt += np.sum(preds == labels.numpy())
        total_cnt += len(labels)

    print("✅ 예측 완료")
    print("총 샘플 수:", total_cnt)
    print("일치 수:", match_cnt)
    print("정확도: {:.2f}%".format((match_cnt / total_cnt) * 100))


def main():
    history_data = None  # history 저장용 변수

    while(True):
        print("1.기본학습")
        print("2.차트")
        print("3.예측")
        print("4.평가하기")
        sel = input("선택 : ")
        if sel == "1":
            history_data = study()
        elif sel == "2":
            try:
                with open("flowers_hist.hist", "rb") as f:
                    history_data = pickle.load(f)
                drawChart(history_data)
            except Exception as e:
                print("❌ 차트를 불러오는 데 실패했습니다:", e)
        elif sel == "3":
            predict()
        elif sel == "4":
            pass
        else:
            return

if __name__ == "__main__":
    main()